In [21]:
import pandas as pd
from datasets import load_dataset
import re
huggingface_ds = load_dataset("chillies/IELTS-writing-task-2-evaluation")['train']
huggingface_ds = pd.DataFrame(huggingface_ds)

In [22]:
evaluation = huggingface_ds["evaluation"]
evaluation.head()
Grammatical_df = pd.DataFrame()
Grammatical_df['prompt'] = huggingface_ds['prompt']
Grammatical_df['essay'] = huggingface_ds['essay']
len(Grammatical_df)

9833

In [23]:
def extract(text):
    start_keyword = "Grammatical Range and Accuracy"
    end_keyword = "Overall Band Score"
    start_index = text.find(start_keyword)
    end_index = text.find(end_keyword)
    if start_index != -1 and end_index != -1:
        return text[start_index + len(start_keyword):end_index].strip()
    else:
        return ""

def clean_text(text):
    cleaned_text = text.replace('*', '').replace('#', '').replace('\n', '').replace('-', '').replace('*', '').strip()
    cleaned_text = cleaned_text.strip(": -")
    cleaned_text = cleaned_text.replace("Suggested Band Score", "").strip()
    cleaned_text = cleaned_text.replace("(Grammatical Range and Accuracy)", "").strip()
    cleaned_text = cleaned_text.strip(": -")
    return cleaned_text

In [24]:
Grammatical_df['text'] = huggingface_ds["evaluation"].apply(extract)
Grammatical_df['text'] = Grammatical_df['text'].apply(clean_text)
Grammatical_df['text'][10]

'4.0 The candidate demonstrates a good variety of sentence structures, including complex sentences. There are a few minor grammatical errors (e.g., "interviews is not a reliable mthod" instead of "interviews are not a reliable method"), but these do not significantly impact the overall quality of the essay. The punctuation and sentence formation are generally correct.'

In [25]:
Grammatical_df['text'][20]

'The candidate demonstrates a limited range of sentence structures, with simple and repetitive sentence patterns. There are frequent grammatical errors, including incorrect verb forms, sentence construction, and punctuation.  : 2.0'

In [26]:
Grammatical_df['text'][30]

'[6]The candidate has demonstrated a good range of grammatical structures and has made few grammatical errors. However, there are some instances where the grammar could be improved. For example, the candidate could have used a more complex sentence structure in the first paragraph.'

In [27]:
empty_string_count = (Grammatical_df['text'] == '').sum()
print(f"Number of rows with empty '': {empty_string_count}")

Number of rows with empty '': 14


In [28]:
Grammatical_df["score"] = Grammatical_df['text'].str.extract(r'(\d+\.?\d*)').astype(float)
Grammatical_df["score"]

0       7.0
1       4.5
2       6.0
3       5.0
4       3.0
       ... 
9828    6.0
9829    4.0
9830    6.5
9831    7.0
9832    6.0
Name: score, Length: 9833, dtype: float64

In [29]:
nan_count = Grammatical_df['score'].isna().sum()
nan_count

62

In [30]:
def clean_text_v2(text):
    cleaned_text = text.replace('*', '').replace('#', '').replace('\n', '').replace('-', '').replace('*', '').strip()
    cleaned_text = re.sub(r"\d+\.*\d*", "", cleaned_text).strip()
    cleaned_text = re.sub(r"\[\d+\.*\d*\]", "", cleaned_text).strip()
    cleaned_text = cleaned_text.strip(": -")
    cleaned_text = cleaned_text.replace("Suggested Band Score", "").strip()
    cleaned_text = cleaned_text.replace("(Grammatical Range and Accuracy)", "").strip()
    cleaned_text = cleaned_text.replace("[]", "").strip()
    return cleaned_text

In [31]:
Grammatical_df['text'] = Grammatical_df['text'].apply(clean_text_v2)

In [32]:
Grammatical_df['text'][10]

'The candidate demonstrates a good variety of sentence structures, including complex sentences. There are a few minor grammatical errors (e.g., "interviews is not a reliable mthod" instead of "interviews are not a reliable method"), but these do not significantly impact the overall quality of the essay. The punctuation and sentence formation are generally correct.'

In [33]:
Grammatical_df['text'][20]

'The candidate demonstrates a limited range of sentence structures, with simple and repetitive sentence patterns. There are frequent grammatical errors, including incorrect verb forms, sentence construction, and punctuation.'

In [34]:
Grammatical_df['text'][30]

'The candidate has demonstrated a good range of grammatical structures and has made few grammatical errors. However, there are some instances where the grammar could be improved. For example, the candidate could have used a more complex sentence structure in the first paragraph.'

In [35]:
Grammatical_df = Grammatical_df[['prompt', 'essay', 'text', 'score']]
Grammatical_df

,prompt,essay,text,score
0,Interviews form the basic criteria for most la...,It is believed by some experts that the tradit...,The essay exhibits a variety of sentence struc...,7.0
1,Interviews form the basic selecting criteria f...,Nowadays numerous huge firms allocate an inter...,The candidate demonstrates a good range of gra...,4.5
2,Interview form the basic selection criteria fo...,The interview section is the most vital part o...,The candidate demonstrates a good grasp of gra...,6.0
3,Interviews form the basic selection criteria f...,It is argued that the best method to recruit e...,The essay exhibits a limited range of sentence...,5.0
4,Interviews from the basic selecting criteria f...,Nowadays many companies conduct interviews bef...,The essay exhibits a narrow range of sentence ...,3.0
...,...,...,...,...
9828,Nations should spend more money on skills and ...,"These days, many countries use a huge amount o...",The essay shows a fair range of sentence struc...,6.0
9829,Nations should spend more money on skills and ...,Skills are required in order to achieve succes...,The essay demonstrates a lack of variety in se...,4.0
9830,Nations should spend more money on skills and ...,There is a contradictory view among the people...,"A variety of sentence structures is used, but ...",6.5
9831,Nations should spend more money on skills and ...,Many today feel that countries should prioriti...,The essay exhibits a variety of sentence struc...,7.0


In [36]:
empty_string_count = (Grammatical_df['text'] == '').sum()
print(f"Number of rows with empty 'text': {empty_string_count}")
Grammatical_df = Grammatical_df[Grammatical_df['text'] != '']

Number of rows with empty 'text': 15


In [37]:
nan_count = Grammatical_df.isna().any(axis=1).sum()
nan_count

48

In [38]:
Grammatical_df = Grammatical_df.dropna(subset=['score'])
nan_count = Grammatical_df.isna().any(axis=1).sum()
nan_count

0

In [39]:
len(Grammatical_df)

9770

In [40]:
csv_path = 'data/Grammatical.csv'
Grammatical_df.to_csv(csv_path, index=False)